In [ ]:
from tensorflow import keras
from matplotlib import pyplot as plt
from metrics import *
from datagenerators import *
from architectures.ResidualUnet import *
from sklearn.model_selection import train_test_split

In [ ]:
model = keras.models.load_model('Models/2D/resU_net_2d_300data.h5', 
                                   custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
                                                   "dice_coef": dice_coef,
                                                   "dice_coef_necrotic": dice_coef_necrotic,
                                                   "dice_coef_edema": dice_coef_edema,
                                                   "dice_coef_enhancing": dice_coef_enhancing
                                                  }, compile=False)

In [ ]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=336, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14)

In [ ]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [ ]:
classes = {
    0 : 'No tumor',
    1 : 'Necrotic region',
    2 : 'Edema region',
    3 : 'Enhancing region'
}


def normalize_intensity(image):
    min_value = np.min(image)
    max_value = np.max(image)
    return (image - min_value) / (max_value - min_value)

def predict_path(case_path,case):
    files = next(os.walk(case_path))
    X = np.empty((VOLUME_SLICES, 128, 128, 3))
    
    vol_path = os.path.join(case_path, f'{case}_flair.nii');
    flair = nib.load(vol_path).get_fdata()
    
    vol_path = os.path.join(case_path, f'{case}_t1ce.nii');
    ce = nib.load(vol_path).get_fdata() 
    
    vol_path = os.path.join(case_path, f'{case}_t2.nii')
    t2 = nib.load(vol_path).get_fdata()
    
    for j in range(VOLUME_SLICES):
        X[j,:,:,0] = cv2.resize(flair[:,:,j+VOLUME_START_AT], (128,128))
        X[j,:,:,1] = cv2.resize(ce[:,:,j+VOLUME_START_AT], (128,128))
        X[j,:,:,2] = cv2.resize(t2[:,:,j+VOLUME_START_AT], (128,128))
        
    return model.predict(X/np.max(X), verbose=1)


def predict_id(case, start_slice = 60):
    path = f'C:/Users/VegardEikenes/Desktop/Bachelor/Data/BraTS2021_TrainingData/{case}'
    gt = nib.load(os.path.join(path, f'{case}_seg.nii')).get_fdata()
    origImage = nib.load(os.path.join(path, f'{case}_flair.nii')).get_fdata()
    p = predict_path(path,case)

    core = p[:,:,:,1]
    edema = p[:,:,:,2]
    enhancing = p[:,:,:,3]

    plt.figure(figsize=(100, 100))
    fig, ax = plt.subplots(1,6, figsize = (25, 25))
    
    normalized_orig_image = normalize_intensity(origImage)
    
    for i in range(6):
        ax[i].imshow(cv2.resize(normalized_orig_image[:,:,start_slice+VOLUME_START_AT], (128, 128)), cmap="gray")
    
    ax[0].imshow(cv2.resize(origImage[:,:,start_slice+VOLUME_START_AT], (128, 128)), cmap="gray")
    ax[0].title.set_text('Original flair image')
    curr_gt=cv2.resize(gt[:,:,start_slice+VOLUME_START_AT], (128, 128), interpolation = cv2.INTER_NEAREST)
    ax[1].imshow(curr_gt, cmap="Reds", alpha=0.3)
    ax[1].title.set_text('Ground truth')
    ax[2].imshow(p[start_slice,:,:,1:4], cmap="Reds", alpha=0.3)
    ax[2].title.set_text('all classes')
    ax[3].imshow(edema[start_slice,:,:], cmap="OrRd", alpha=0.3)
    ax[3].title.set_text(f'Predicted {classes[1]}')
    ax[4].imshow(core[start_slice,:,:], cmap="OrRd", alpha=0.3)
    ax[4].title.set_text(f'Predicted {classes[2]}')
    ax[5].imshow(enhancing[start_slice,:,:], cmap="OrRd", alpha=0.3)
    ax[5].title.set_text(f'Predicted {classes[3]}')
    # plt.savefig('BraTS2021_00590.png')
    
    
predict_id(case=test_ids[0])

In [ ]:
test_ids[0]